In [1]:
pip install neuralforecast transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import yfinance as yf
import pandas as pd


ticker = 'AAPL'
start_date = '1900-01-01'
end_date = '2023-12-31'

# دانلود داده‌ها
data = yf.download(ticker, start=start_date, end=end_date)

# نمایش داده‌ها
print(data)


[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1980-12-12    0.128348    0.128906    0.128348    0.128348    0.098943   
1980-12-15    0.122210    0.122210    0.121652    0.121652    0.093781   
1980-12-16    0.113281    0.113281    0.112723    0.112723    0.086898   
1980-12-17    0.115513    0.116071    0.115513    0.115513    0.089049   
1980-12-18    0.118862    0.119420    0.118862    0.118862    0.091630   
...                ...         ...         ...         ...         ...   
2023-12-22  195.179993  195.410004  192.970001  193.600006  192.868134   
2023-12-26  193.610001  193.889999  192.830002  193.050003  192.320221   
2023-12-27  192.490005  193.500000  191.089996  193.149994  192.419830   
2023-12-28  194.139999  194.660004  193.169998  193.580002  192.848221   
2023-12-29  193.899994  194.399994  191.729996  192.529999  191.802170   

               Volume  
Date         

In [4]:

data.reset_index(inplace=True)
data.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

Y_train_df = data[data['ds'] < '2023-01-01']
Y_test_df = data[data['ds'] >= '2023-01-01'].reset_index(drop=True)


In [5]:
# افزودن ستون 'unique_id' به دیتافریم
Y_train_df['unique_id'] = 'AAPL'  # یا هر شناسه‌ای که مناسب است
Y_test_df['unique_id'] = 'AAPL'

<ipython-input-5-d3066d87313b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train_df['unique_id'] = 'AAPL'  # یا هر شناسه‌ای که مناسب است


In [6]:
from transformers import GPT2Config, GPT2Model, GPT2Tokenizer
from neuralforecast.models import TimeLLM

# Set up GPT-2 model, tokenizer, and configuration
gpt2_config = GPT2Config.from_pretrained('openai-community/gpt2')
gpt2 = GPT2Model.from_pretrained('openai-community/gpt2', config=gpt2_config)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2')

# Define the model
prompt_prefix = "Historical stock price data of Apple Inc., recorded daily over a period of more than four decades. This dataset includes various financial metrics and indicators. It contains data on 10,000+ daily observations collected from the NASDAQ stock exchange between December 12, 1980, and December 31, 2023 (approximately 43 years)."
timellm = TimeLLM(
    h=1,  # Prediction length of one
    input_size=64,  # Adjust based on your data
    llm=gpt2,
    llm_config=gpt2_config,
    llm_tokenizer=gpt2_tokenizer,
    prompt_prefix=prompt_prefix,
    batch_size=1,
    windows_batch_size=1,
    enable_checkpointing=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

INFO:lightning_fabric.utilities.seed:Seed set to 1


In [7]:
from neuralforecast import NeuralForecast

nf = NeuralForecast(
    models=[timellm],
    freq='D'
)

nf.fit(df=Y_train_df, val_size=12)


/usr/local/lib/python3.10/dist-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type               | Params | Mode 
-------------------------------------------------------------------
0 | loss                | MAE                | 0      | train
1 | padder_train        | ConstantPad1d      | 0      | train
2 | scaler              | TemporalNorm       | 0      | train
3 | llm                 | GPT2Model          | 124 M  | eval 
4 | patch_embedding     | PatchEmbedding     | 1.5 K 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5` reached.


In [8]:
forecasts = nf.predict()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:209: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [11]:
Y_train_df

,ds,Open,High,Low,y,Adj Close,Volume,unique_id
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,AAPL
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,AAPL
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,AAPL
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,AAPL
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,AAPL
...,...,...,...,...,...,...,...,...
10598,2022-12-23,130.919998,132.419998,129.639999,131.860001,130.631378,63814900,AAPL
10599,2022-12-27,131.380005,131.410004,128.720001,130.029999,128.818390,69007800,AAPL
10600,2022-12-28,129.669998,131.029999,125.870003,126.040001,124.865585,85438400,AAPL
10601,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.402313,75703700,AAPL


In [9]:
forecasts

,ds,TimeLLM
unique_id,,
AAPL,2022-12-31,139.191177


In [12]:
nf.save(path='./checkpoints/',
        model_index=None,
        overwrite=True,
        save_dataset=True)
